# 模型，提示和输出解释器


**目录**
* 获取你的OpenAI API Key
* 直接调用OpenAI的API
* 通过LangChain进行的API调用：
  * 提示（Prompts）
  * [模型（Models)](#model)
  * 输出解析器（Output parsers）
  

## 获取你的OpenAI API Key

登陆[OpenAI账户获取你的API Key](https://platform.openai.com/account/api-keys) 

In [1]:
# 下载需要的包python-dotenv和openai
# 如果你需要查看安装过程日志，可删除 -q 
!pip install -q python-dotenv
!pip install -q openai


In [1]:
# 将自己的 API-KEY 导入系统环境变量（关于如何设置参考这篇文章：https://zhuanlan.zhihu.com/p/627665725）
!export OPENAI_API_KEY='api-key'     #api_key替换为自己的

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地的环境变量 
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']  

## Chat API：OpenAI

我们先从直接调用OpenAI的API开始。

`get_completion`函数是基于`openai`的封装函数，对于给定提示（prompt）输出相应的回答。其包含两个参数
   
   - `prompt` 必需输入参数。 你给模型的**提示，可以是一个问题，可以是你需要模型帮助你做的事**（改变文本写作风格，翻译，回复消息等等）。
   - `model` 非必需输入参数。默认使用gpt-3.5-turbo。你也可以选择其他模型。
   
这里的提示对应我们给chatgpt的问题，函数给出的输出则对应chatpgt给我们的答案。

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

### 一个简单的例子

我们来一个简单的例子 - 分别用中英文问问模型

- 中文提示(Prompt in Chinese)： `1+1是什么？`
- 英文提示(Prompt in English)： `What is 1+1?`

In [ ]:
# 中文
get_completion("1+1是什么？")

'1+1等于2。'

In [5]:
# 英文
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

### 复杂一点例子

上面的简单例子，模型`gpt-3.5-turbo`对我们的关于1+1是什么的提问给出了回答。

现在我们来看一个复杂一点的例子： 

假设我们是电商公司员工，我们的顾客是一名海盗A，他在我们的网站上买了一个榨汁机用来做奶昔，在制作奶昔的过程中，奶昔的盖子飞了出去，弄得厨房墙上到处都是。于是海盗A给我们的客服中心写来以下邮件：`customer_email`

In [6]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

我们的客服人员对于海盗的措辞表达觉得有点难以理解。 现在我们想要实现两个小目标：

- 让模型用美式英语的表达方式将海盗的邮件进行翻译，客服人员可以更好理解。*这里海盗的英文表达可以理解为英文的方言，其与美式英语的关系，就如四川话与普通话的关系。
- 让模型在翻译是用平和尊重的语气进行表达，客服人员的心情也会更好。

根据这两个小目标，定义一下文本表达风格：`style`

In [7]:
# 美式英语 + 平静、尊敬的语调
style = """American English \
in a calm and respectful tone
"""

下一步需要做的是将`customer_email`和`style`结合起来构造我们的提示:`prompt`

In [8]:
# 要求模型根据给出的语调进行转化
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



`prompt` 构造好了，我们可以调用`get_completion`得到我们想要的结果 - 用平和尊重的语气，美式英语表达的海岛邮件

In [ ]:
response = get_completion(prompt)

In [10]:
response

'I am quite upset that my blender lid came off and caused my smoothie to splatter all over my kitchen walls. Additionally, the warranty does not cover the cost of cleaning up the mess. Would you be able to assist me, please? Thank you kindly.'

对比语言风格转换前后，用词更为正式，替换了极端情绪的表达，并表达了感谢。
- Arrr, I be fuming（呀，我气的发抖） 换成了 I am quite upset （我有点失望）
- And to make matters worse（更糟糕地是），换成了 Additionally(还有)
- I need yer help right now, matey!（我需要你的帮助），换成了Would you be able to assist me, please? Thank you kindly.（请问您能帮我吗？非常感谢您的好意）


✨ 你可以尝试修改提示，看可以得到什么不一样的结果😉

### 中文

假设我们是电商公司员工，我们的顾客是一名海盗A，他在我们的网站上买了一个榨汁机用来做奶昔，在制作奶昔的过程中，奶昔的盖子飞了出去，弄得厨房墙上到处都是。于是海盗A给我们的客服中心写来以下邮件：`customer_email`

In [3]:
# 非正式用语
customer_email = """   
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

我们的客服人员对于海盗的措辞表达觉得有点难以理解。 现在我们想要实现两个小目标：

- 让模型用比较正式的普通话的表达方式将海盗的邮件进行翻译，客服人员可以更好理解。
- 让模型在翻译是用平和尊重的语气进行表达，客服人员的心情也会更好。

根据这两个小目标，定义一下文本表达风格：`style`

In [8]:
# 普通话 + 平静、尊敬的语调
style = """正式普通话 \
用一个平静、尊敬的语调
"""

下一步需要做的是将`customer_email`和`style`结合起来构造我们的提示:`prompt`

In [9]:
# 要求模型根据给出的语调进行转化
prompt = f"""把由三个反引号分隔的文本text\
翻译成一种{style}风格。
text: ```{customer_email}```
"""

print(prompt)

把由三个反引号分隔的文本text翻译成一种正式普通话 用一个平静、尊敬的语调
风格。
text: ```   
阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！
```



`prompt` 构造好了，我们可以调用`get_completion`得到我们想要的结果 - 用平和尊重的普通话语气去表达一封带着方言表达方式的邮件

In [10]:
response = get_completion(prompt)

In [11]:
response

'尊敬的朋友，我感到不安，因为我的搅拌机盖子不慎掉落，导致奶昔溅到了厨房的墙壁上！更加令人糟心的是，保修并不包含厨房清洁的费用。此刻，我需要你的帮助，亲爱的朋友！'

## Chat API：LangChain

在前面一部分，我们通过封装函数`get_completion`直接调用了OpenAI完成了对方言邮件进行了的翻译，得到用平和尊重的语气、正式的普通话表达的邮件。

让我们尝试使用LangChain来实现相同的功能。

In [14]:
# 如果你需要查看安装过程日志，可删除 -q 
# --upgrade 让我们可以安装到最新版本的 langchain
!pip install -q --upgrade langchain

^C


### 模型<a id='model'></a>

从`langchain.chat_models`导入`OpenAI`的对话模型`ChatOpenAI`。 除去OpenAI以外，`langchain.chat_models`还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://python.langchain.com/en/latest/modules/models/chat/integrations.html)。

In [15]:
from langchain.chat_models import ChatOpenAI

In [13]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

上面的输出显示ChatOpenAI的默认模型为`gpt-3.5-turbo`

### 提示模板

在前面的例子中，我们通过[f字符串](https://docs.python.org/zh-cn/3/tutorial/inputoutput.html#tut-f-strings)把Python表达式的值`style`和`customer_email`添加到`prompt`字符串内。

```python
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""
```
`langchain`提供了接口方便快速的构造和使用提示。现在我们来看看如何使用`langchain`来构造提示。

#### 📚 使用LangChain提示模版
##### 1️⃣ 构造提示模版字符串
我们构造一个提示模版字符串：`template_string`

In [8]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [17]:
# 中文
template_string = """把由三个反引号分隔的文本text\
翻译成一种{style}风格。\
text: ```{text}```
"""

##### 2️⃣ 构造LangChain提示模版
我们调用`ChatPromptTemplatee.from_template()`函数将上面的提示模版字符`template_string`转换为提示模版`prompt_template`

In [18]:
# 需要安装最新版的 LangChain
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [10]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [19]:
# 中文
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='把由三个反引号分隔的文本text翻译成一种{style}风格。text: ```{text}```\n', template_format='f-string', validate_template=True)

从上面的输出可以看出，`prompt_template` 有两个输入变量： `style` 和 `text`。

In [7]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

##### 3️⃣ 使用模版得到客户消息提示

langchain提示模版`prompt_template`需要两个输入变量： `style` 和 `text`。 这里分别对应 
- `customer_style`: 我们想要的顾客邮件风格
- `customer_email`: 顾客的原始邮件文本。

In [11]:
customer_style = """American English \
in a calm and respectful tone
"""

In [20]:
# 中文
customer_style = """正式普通话 \
用一个平静、尊敬的语气
"""

In [12]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [21]:
# 中文
customer_email = """
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

对于给定的`customer_style`和`customer_email`, 我们可以使用提示模版`prompt_template`的`format_messages`方法生成想要的客户消息`customer_messages`。

In [22]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [11]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


可以看出`customer_messages`变量类型为列表(`list`)，而列表里的元素变量类型为langchain自定义消息(`langchain.schema.HumanMessage`)。

打印第一个元素可以得到如下:

In [12]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


In [24]:
# 中文
print(customer_messages[0])

content='把由三个反引号分隔的文本text翻译成一种正式普通话 用一个平静、尊敬的语气\n风格。text: ```\n阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！\n```\n' additional_kwargs={} example=False


##### 4️⃣ 调用chat模型转换客户消息风格

现在我们可以调用[模型](#model)部分定义的chat模型来实现转换客户消息风格。到目前为止，我们已经实现了在前一部分的任务。

In [25]:
customer_response = chat(customer_messages)

In [15]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add insult to injury, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?


In [26]:
print(customer_response.content)

尊敬的伙计，我感到非常不安，因为我的搅拌机盖子不慎掉落，导致奶昔溅到了厨房的墙壁上！更加令人糟心的是，保修并不包括厨房清洁的费用。现在，我非常需要您的帮助！


##### 5️⃣ 使用模版得到回复消息提示

接下来，我们更进一步，将客服人员回复的消息，转换为海盗的语言风格，并确保消息比较有礼貌。 

这里，我们可以继续使用第2️⃣步构造的langchain提示模版，来获得我们回复消息提示。

In [16]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [27]:
# 中文
service_reply = """嘿，顾客， \
保修不包括厨房的清洁费用， \
因为您在启动搅拌机之前 \
忘记盖上盖子而误用搅拌机, \
这是您的错。 \
倒霉！ 再见！
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [28]:
# 中文
service_style_pirate = """\
一个有礼貌的语气 \
使用正式的普通话\
"""

In [18]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [29]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

把由三个反引号分隔的文本text翻译成一种一个有礼貌的语气 使用正式的普通话风格。text: ```嘿，顾客， 保修不包括厨房的清洁费用， 因为您在启动搅拌机之前 忘记盖上盖子而误用搅拌机, 这是您的错。 倒霉！ 再见！
```



##### 6️⃣ 调用chat模型转换回复消息风格

调用[模型](#model)部分定义的chat模型来转换回复消息风格

In [19]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, me hearty customer! I be sorry to inform ye that the warranty be not coverin' the expenses o' cleaning yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Farewell and may the winds be in yer favor!


In [30]:
service_response = chat(service_messages)
print(service_response.content)

尊敬的顾客，根据保修条款，厨房清洁费用不在保修范围内。这是因为在使用搅拌机之前，您忘记盖上盖子而导致误用搅拌机，这属于您的疏忽。非常抱歉给您带来困扰！祝您好运！再见！


#### ❓为什么需要提示模版





在应用于比较复杂的场景时，提示可能会非常长并且包含涉及许多细节。**使用提示模版，可以让我们更为方便地重复使用设计好的提示**。

下面给出了一个比较长的提示模版案例。学生们线上学习并提交作业，通过以下的提示来实现对学生的提交的作业的评分。

```python
    prompt = """ Your task is to determine if the student's solution is correct or not

    To solve the problem do the following:
    - First, workout your own solution to the problem
    - Then compare your solution to the student's solution 
    and evaluate if the sudtent's solution is correct or not.
    ...
    Use the following format:
    Question:
    ```
    question here
    ```
    Student's solution:
    ```
    student's solution here
    ```
    Actual solution:
    ```
    ...
    steps to work out the solution and your solution here
    ```
    Is the student's solution the same as acutal solution \
    just calculated:
    ```
    yes or no
    ```
    Student grade
    ```
    correct or incorrect
    ```
    
    Question:
    ```
    {question}
    ```
    Student's solution:
    ```
    {student's solution}
    ```
    Actual solution:
    
    """
```

# 中文

```python
    prompt = """ 你的任务是判断学生的解决方案是正确的还是不正确的

    要解决该问题，请执行以下操作：
     - 首先，制定自己的问题解决方案
     - 然后将您的解决方案与学生的解决方案进行比较
     并评估学生的解决方案是否正确。
    ...
    使用下面的格式:

    问题:
    ```
    问题文本
    ```
    学生的解决方案:
    ```
    学生的解决方案文本
    ```
    实际解决方案:
    ```
    ...
    制定解决方案的步骤以及您的解决方案请参见此处
    ```
    学生的解决方案和实际解决方案是否相同 \
    只计算：
    ```
    是或者不是
    ```
    学生的成绩
    ```
    正确或者不正确
    ```
    
    问题:
    ```
    {question}
    ```
    学生的解决方案:
    ```
    {student's solution}
    ```
    实际解决方案:
    
    """
```



此外，LangChain还提供了提示模版用于一些常用场景。比如summarization, Question answering, or connect to sql databases, or connect to different APIs. 通过使用LongChain内置的提示模版，你可以快速建立自己的大模型应用，而不需要花时间去设计和构造提示。

最后，我们在建立大模型应用时，通常希望模型的输出为给定的格式，比如在输出使用特定的关键词来让输出结构化。 下面为一个使用大模型进行链式思考推理例子，对于问题：`What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?`, 通过使用LangChain库函数，输出采用"Thought"（思考）、"Action"（行动）、"Observation"（观察）作为链式思考推理的关键词，让输出结构化。在[补充材料](#reason_act)中，可以查看使用LangChain和OpenAI进行链式思考推理的另一个代码实例。

```python
"""
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions

Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]

Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

"""
```


```python
"""
想法：我需要搜索科罗拉多造山带，找到科罗拉多造山带东段延伸到的区域，然后找到该区域的高程范围。
行动：搜索[科罗拉多造山运动]
观察：科罗拉多造山运动是科罗拉多州及周边地区造山运动（造山运动）的一次事件。

想法：它没有提到东区。 所以我需要查找东区。
行动：查找[东区]
观察：（结果1 / 1）东段延伸至高原，称为中原造山运动。

想法：科罗拉多造山运动的东段延伸至高原。 所以我需要搜索高原并找到它的海拔范围。
行动：搜索[高地平原]
观察：高原是指两个不同的陆地区域之一

想法：我需要搜索高地平原（美国）。
行动：搜索[高地平原（美国）]
观察：高地平原是大平原的一个分区。 从东到西，高原的海拔从 1,800 英尺左右上升到 7,000 英尺（550 到 2,130 米）。[3]

想法：高原的海拔从大约 1,800 英尺上升到 7,000 英尺，所以答案是 1,800 到 7,000 英尺。
动作：完成[1,800 至 7,000 英尺]

"""
```


### 输出解析器

#### 📚 如果没有输出解析器

对于给定的评价`customer_review`, 我们希望提取信息，并按以下格式输出：

```python
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [20]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [31]:
# 中文
customer_review = """\
这款吹叶机非常神奇。 它有四个设置：\
吹蜡烛、微风、风城、龙卷风。 \
两天后就到了，正好赶上我妻子的\
周年纪念礼物。 \
我想我的妻子会喜欢它到说不出话来。 \
到目前为止，我是唯一一个使用它的人，而且我一直\
每隔一天早上用它来清理草坪上的叶子。 \
它比其他吹叶机稍微贵一点，\
但我认为它的额外功能是值得的。
"""

##### 1️⃣ 构造提示模版字符串

In [21]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [32]:
# 中文
review_template = """\
对于以下文本，请从中提取以下信息：

礼物：该商品是作为礼物送给别人的吗？ \
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品需要多少天\
到达？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，\
并将它们输出为逗号分隔的 Python 列表。

使用以下键将输出格式化为 JSON：
礼物
交货天数
价钱

文本: {text}
"""

##### 2️⃣ 构造langchain提示模版

In [22]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [33]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='对于以下文本，请从中提取以下信息：\n\n礼物：该商品是作为礼物送给别人的吗？ 如果是，则回答 是的；如果否或未知，则回答 不是。\n\n交货天数：产品需要多少天到达？ 如果没有找到该信息，则输出-1。\n\n价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。\n\n使用以下键将输出格式化为 JSON：\n礼物\n交货天数\n价钱\n\n文本: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


##### 3️⃣ 使用模版得到提示消息

In [34]:
messages = prompt_template.format_messages(text=customer_review)

##### 4️⃣ 调用chat模型提取信息

In [24]:
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [36]:
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
  "礼物": "是的",
  "交货天数": 2,
  "价钱": ["它比其他吹叶机稍微贵一点"]
}


##### 📝 分析与总结
`response.content`类型为字符串（`str`），而并非字典(`dict`), 直接使用`get`方法会报错。因此，我们需要输出解释器。

In [34]:
type(response.content)

str

In [35]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

#### 📚 LangChain输出解析器

##### 1️⃣ 构造提示模版字符串

In [36]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [45]:
# 中文
review_template_2 = """\
对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: {text}

{format_instructions}
"""

##### 2️⃣ 构造langchain提示模版

In [46]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

##### 🔥 构造输出解析器

In [38]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")


response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)



The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [47]:
# 中文
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="礼物",
                             description="这件物品是作为礼物送给别人的吗？\
                            如果是，则回答 是的，\
                            如果否或未知，则回答 不是。")

delivery_days_schema = ResponseSchema(name="交货天数",
                                      description="产品需要多少天才能到达？\
                                      如果没有找到该信息，则输出-1。")

price_value_schema = ResponseSchema(name="价钱",
                                    description="提取有关价值或价格的任何句子，\
                                    并将它们输出为逗号分隔的 Python 列表")


response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```


##### 3️⃣ 使用模版得到提示消息

In [48]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [40]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "

In [43]:
# 中文
print(messages[0].content)

对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: 这款吹叶机非常神奇。 它有四个设置：吹蜡烛、微风、风城、龙卷风。 两天后就到了，正好赶上我妻子的周年纪念礼物。 我想我的妻子会喜欢它到说不出话来。 到目前为止，我是唯一一个使用它的人，而且我一直每隔一天早上用它来清理草坪上的叶子。 它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```



##### 4️⃣ 调用chat模型提取信息

In [41]:
response = chat(messages)
print(response.content)

```json
{
	"gift": true,
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [49]:
# 中文
response = chat(messages)
print(response.content)

```json
{
	"礼物": "不是",
	"交货天数": "两天后就到了",
	"价钱": "它比其他吹叶机稍微贵一点"
}
```


##### 5️⃣ 使用输出解析器解析输出

In [42]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': True,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [50]:
output_dict = output_parser.parse(response.content)
output_dict

{'礼物': '不是', '交货天数': '两天后就到了', '价钱': '它比其他吹叶机稍微贵一点'}

##### 📝 分析与总结
`output_dict`类型为字典(`dict`), 可直接使用`get`方法。这样的输出更方便下游任务的处理。

In [43]:
type(output_dict)

dict

In [44]:
output_dict.get('delivery_days')

'2'

## 补充材料

### 链式思考推理(ReAct) <a id='reason_act'></a>
参考资料：[ReAct (Reason+Act) prompting in OpenAI GPT and LangChain](https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/)

In [ ]:
!pip install -q wikipedia

In [56]:
from langchain.docstore.wikipedia import Wikipedia
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  )
]

# 使用大语言模型
llm = OpenAI(
  model_name="gpt-3.5-turbo",
  temperature=0,
)

# 初始化ReAct代理
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
)


question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search David Chanoff and find the U.S. Navy admiral he collaborated with. Then I need to find the president under whom the admiral served as the ambassador to the United Kingdom.
Action: Search[David Chanoff]
Observation: David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.
Thought:David Chanoff has collaborated with several individuals, including Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good, and Felix Zandman. I need to

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-kmMHlitaQynVMwTW3jDTCPga on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-kmMHlitaQynVMwTW3jDTCPga on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/bi

Đoàn Văn Toại is an activist and author, not a U.S. Navy admiral. I need to search the next individual, William J. Crowe.
Action: Search[William J. Crowe]
Observation: William James Crowe Jr. (January 2, 1925 – October 18, 2007) was a United States Navy admiral and diplomat who served as the 11th chairman of the Joint Chiefs of Staff under Presidents Ronald Reagan and George H. W. Bush, and as the ambassador to the United Kingdom and Chair of the Intelligence Oversight Board under President Bill Clinton.
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-kmMHlitaQynVMwTW3jDTCPga on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-kmMHlitaQynVMwTW3jDTCPga on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/bi

William J. Crowe is a U.S. Navy admiral and diplomat. I need to find the president under whom he served as the ambassador to the United Kingdom.
Action: Search[William J. Crowe ambassador to United Kingdom]
Observation: William James Crowe Jr. (January 2, 1925 – October 18, 2007) was a United States Navy admiral and diplomat who served as the 11th chairman of the Joint Chiefs of Staff under Presidents Ronald Reagan and George H. W. Bush, and as the ambassador to the United Kingdom and Chair of the Intelligence Oversight Board under President Bill Clinton.
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-kmMHlitaQynVMwTW3jDTCPga on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-kmMHlitaQynVMwTW3jDTCPga on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/bi

William J. Crowe served as the ambassador to the United Kingdom under President Bill Clinton. So the answer is Bill Clinton.
Action: Finish[Bill Clinton]

> Finished chain.


'Bill Clinton'